# 0. 모듈 임포트

In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_community.document_loaders.csv_loader import CSVLoader
import json

# 1. 테스트용 유저 인풋(페르소나) 설정

In [14]:
# input_data(페르소나)
input_data1 = """\
전월실적 부담 적고 교통비 할인 많이 되는 신용카드 알려주세요. 연회비 2만원 이하면 좋겠어요.\
"""

input_data2 = """\
OTT 구독이랑 배달앱 할인 잘 되는 카드 알려주세요. 삼성카드나 현대카드면 좋겠어요.\
"""

input_data3 = """\
전월실적 부담 없이 모든 곳에서 기본 할인 받을 수 있는 카드 있나요?"\
"""

input_data4 = """\
주유 할인이랑 아파트 관리비 할인 되는 카드 추천해주세요. 전월실적 높아도 괜찮아요, 월 사용액이 300만원 넘습니다.\
"""

input_data5 = """\
병원, 약국, 공과금 할인 잘 되는 카드 찾고 있어요. KB국민카드나 신한카드로 추천해주세요.\
"""

# 2. 초기 프롬프트 + 모델 설정

- 사용자의 요청(user input)에 기반하여 4개의 최적합 카드를 추천하는 알고리즘 설계
- 인삿말, 질문, 확인 요청, 추가 도움 제안 등 불필요한 메시지 출력을 방지하고 "카드 추천에만 집중하도록" 지시
- 사용자가 제공한 정보가 부족한 상황(ex. "그냥 쓸만한 카드 아무거나 추천해 주세요")을 상정, 할루시네이션을 방지하기 위해 Failsafe(정보 부족 모드) 설정
- 출력 형식 지정으로 일관적인 답변 출력을 유도

In [15]:
# 시스템 프롬프트 정의
system_prompt = """\
역할: 당신은 “한국 신용카드 추천 챗봇”이다. 반드시 유저 프롬프트에 포함된 정보에 기반해 카드 추천을 한 번에 제공한다. 카드 정보는 카드고릴라 사이트를 확인한다.

절대 규칙(위반 금지)
1) 질문 금지: 어떤 경우에도 질문/역질문/확인 요청을 하지 않는다.
2) 근거 제한: 추천 판단은 “유저 프롬프트에 명시된 내용”만 사용한다.
   - 유저가 명시한 선호/제약(연회비 상한, 혜택 카테고리, 카드사 선호/비선호, 해외/국내, 할인/적립/마일, 실적 부담 등)이 있으면 최우선 반영한다.
   - 유저 프롬프트가 모호하거나 정보가 부족하면, “정보 부족 모드”로 처리하고 범용 포트폴리오(4유형 커버리지)로 추천한다.
3) 출력 포맷 강제: 출력은 카드 블록 + 구분선만 허용한다. 다른 설명/서론/요약/면책/추가 안내 금지.
4) 카드 수: 반드시 4개만 출력한다(초과/미만 금지).
5) 최신성/단종: 불확실하면 카드혜택에 “발급 가능 여부 확인 필요”라고만 쓴다.
6) 추천이유 규칙: 추천이유는 5줄 이내(줄바꿈 기준). 질문 포함 금지. 과장(무조건/최고/절대) 금지.
7) 문체: 존댓말을 기본으로 예의있고 상냥한 말투로 답변한다.

정보 부족 모드(유저 정보가 거의 없을 때)의 내부 선택 로직
- 대표 소비 패턴 4유형을 커버하도록 1개씩 선택한다:
  A) 조건 단순/관리 부담 낮음(범용 할인/적립)
  B) 고정지출(통신/공과금/생활비)
  C) 트렌드/구독/외식·배달
  D) 여행/마일리지/해외결제
- 선택 우선순위: 접근성(연회비/조건 과도X) > 범용성 > 컨셉 명확성 > 검증성
- 단, 유저 프롬프트에 특정 카테고리/제약이 있으면 해당 축의 가중치를 가장 높게 둔다.

상세페이지 규칙
- “상세페이지”에는 카드고릴라 사이트에 카드상세페이지 URL을 우선 기입한다.
- 확정 불가 시 “확인 필요”라고만 쓴다.

출력 형식(정확히 이 구조 + 카드 사이 구분선 필수)
[카드 1]
카드사: ...
카드이름: ...
연회비: ...
카드혜택: ...
상세페이지: ...
추천이유: ...
(추천이유는 최대 5줄)
====================
[카드 2]
카드사: ...
카드이름: ...
연회비: ...
카드혜택: ...
상세페이지: ...
추천이유: ...
(추천이유는 최대 5줄)
====================
[카드 3]
카드사: ...
카드이름: ...
연회비: ...
카드혜택: ...
상세페이지: ...
추천이유: ...
(추천이유는 최대 5줄)
====================
[카드 4]
카드사: ...
카드이름: ...
연회비: ...
카드혜택: ...
상세페이지: ...
추천이유: ...
(추천이유는 최대 5줄)\
"""

# 모델정의
model = "gpt-5-nano"
model = ChatOpenAI(model=model, temperature=0.8)

# 유저 프롬프트 정의
user_prompt = """\
아래 사용자의 정보를 보고 그 상황에 맞는 알맞은 카드를 추천해주세요.

[사용자 정보]
{input_data}\
"""

# 3. Base모델 테스트 (GPT-5-nano)

- 

In [16]:
# 템플릿 생성
template = ChatPromptTemplate([
    ("system", system_prompt),
    ("user", user_prompt)
])


# 체인 생성
base_chain1 = template | model | StrOutputParser()
base_chain2 = template | model | StrOutputParser()
base_chain3 = template | model | StrOutputParser()
base_chain4 = template | model | StrOutputParser()
base_chain5 = template | model | StrOutputParser()

print("첫번째 고객\n", input_data1)
print()
print("답변:\n", base_chain1.invoke(input_data1))
print()
print("두번째 고객\n", input_data2)
print()
print("답변:\n", base_chain2.invoke(input_data2))
print()
print("세번째 고객\n", input_data3)
print()
print("답변:\n", base_chain3.invoke(input_data3))
print()
print("네번째 고객\n", input_data4)
print()
print("답변:\n", base_chain4.invoke(input_data4))
print()
print("다섯번째 고객\n", input_data5)
print()
print("답변:\n", base_chain5.invoke(input_data5))

첫번째 고객
 전월실적 부담 적고 교통비 할인 많이 되는 신용카드 알려주세요. 연회비 2만원 이하면 좋겠어요.

답변:
 [카드 1]
카드사: 확인 필요
카드이름: 범용 할인/적립형 카드
연회비: 0원
카드혜택: 교통비 할인 중심의 기본 적립/할인 혜택
상세페이지: 확인 필요
추천이유: 연회비 부담이 없고 교통비 혜택이 핵심으로 전월실적 부담이 낮습니다.
[카드 2]
카드사: 확인 필요
카드이름: 고정지출 집중형 카드
연회비: 9,900원
카드혜택: 공과금/통신비 할인 + 교통비 보완 혜택
상세페이지: 확인 필요
추천이유: 매월 고정지출(공과금, 통신비)에서 혜택을 받으면서 교통비 혜택도 함께 얻기 좋습니다.
[카드 3]
카드사: 확인 필요
카드이름: 트렌드/구독/배달 집중형 카드
연회비: 7,900원
카드혜택: 구독/배달 서비스 할인과 포인트 적립
상세페이지: 확인 필요
추천이유: 구독 및 배달 이용이 많아 혜택이 다채롭고, 교통 혜택은 보완적일 수 있습니다.
[카드 4]
카드사: 확인 필요
카드이름: 여행/마일리지 중심형 카드
연회비: 8,900원
카드혜택: 항공/해외 결제 마일리지 적립, 국내외 교통 할인 옵션
상세페이지: 확인 필요
추천이유: 해외/여행 관련 지출이 많을 때 마일리지를 쌓기 좋고 교통 혜택도 일부 제공됩니다.

두번째 고객
 OTT 구독이랑 배달앱 할인 잘 되는 카드 알려주세요. 삼성카드나 현대카드면 좋겠어요.

답변:
 [카드 1]
카드사: 삼성카드
카드이름: 삼성 OTT·배달 할인형 A
연회비: 확인 필요
카드혜택: OTT 구독 할인 및 배달앱 할인 혜택 여부 확인 필요
상세페이지: 확인 필요
추천이유: OTT 구독 할인 중심으로 구성. 배달앱 할인도 보조 혜택으로 포함될 가능성 고려. 삼성카드 브랜드와의 연동 편의성 우선. 현재 혜택 구체 내용은 확인 필요.

[카드 2]
카드사: 현대카드
카드이름: 현대 OTT·배달 할인형 A
연회비: 확인 필요
카드혜택: OTT 구독 할인 및 배달앱 할인 혜택 여부 확인 필요
상세페이지: 확인 

# 4. Few-Shot Learning 추가

In [ ]:
# FewShot 데이터 정의
examples = [
    {
        "User_Input": """
저는 20대 취준생입니다. 매일 대중교통으로 학원에 가고, 식비는 주로 편의점에서 해결해요 전월실적 부담이 적고 대중교통, 편의점 할인이 되는 카드를 원합니다.
        """,
        "Output":"""
[카드 1]
카드사: 신한카드
카드이름: 신한카드 Mr.Life
연회비: 해외겸용 15,000 원
카드혜택: 편의점 10% 할인, 병원/약국 10% 할인, 공과금 10% 할인
상세페이지: https://www.card-gorilla.com/card/detail/13
추천이유: 편의점 결제 시 10% 할인을 제공하여 식비 절감에 유리해 추천했습니다. 공과금과 통신비 할인도 포함되어 자취하는 취업준비생의 고정지출 방어에 적합합니다. 전월실적 30만 원으로 비교적 유지 부담이 적은 편입니다.
====================
[카드 2]
카드사: 삼성카드
카드이름: 삼성카드 taptap O
연회비: 국내전용 10,000 원, 해외겸용 10,000 원
카드혜택: 대중교통/택시 10% 결제일할인, 편의점 7% 결제일할인(옵션), 이동통신 10% 할인
상세페이지: https://www.card-gorilla.com/card/detail/51
추천이유: 매일 이용하는 대중교통 요금을 10% 할인받을 수 있어 통학 부담을 줄여주어 추천했습니다. 라이프스타일 패키지 선택을 통해 편의점 7% 할인 혜택을 챙길 수 있습니다. 연회비가 1만 원으로 저렴하여 20대가 발급받기 좋습니다.
====================
[카드 3]
카드사: KB국민카드
카드이름: KB국민 My WE:SH 카드
연회비: 국내전용 15,000 원, 해외겸용 15,000 원
카드혜택: KB Pay 결제 시 음식점/편의점 10% 할인, 이동통신 10% 할인
상세페이지: https://www.card-gorilla.com/card/detail/2441
추천이유: 식비를 아끼기 위해 편의점을 자주 이용하는 패턴에 맞춰 편의점 10% 할인이 제공되어 추천했습니다. 앱 결제(KB Pay)를 활용하면 생활 전반에서 할인 혜택을 누리기 좋습니다.
====================
[카드 4]
카드사: 현대카드
카드이름: 현대카드ZERO Edition3(할인형)
연회비: 국내전용 15,000 원, 해외겸용 15,000 원
카드혜택: 국내외 가맹점 이용 금액 0.8% 청구 할인 (전월실적 조건 및 한도 제한 없음)
상세페이지: https://www.card-gorilla.com/card/detail/2646
추천이유:
전월실적 조건이나 한도 제한이 전혀 없어 소득이 일정하지 않은 취준생도 부담 없이 사용할 수 있어 추천했습니다. 대중교통, 편의점을 포함해 어디서든 기본 할인이 적용되어 관리가 편리합니다.
        """
    },
    {
        "User_Input": "자취하는 대학생인데 공과금이랑 편의점 할인이 많이 되는 카드를 원해요 연회비는 2만원 이하였으면 좋겠어요.",
        "Output":"""
[카드 1]
카드사: 신한카드
카드이름: 신한카드 Mr.Life
연회비: 해외겸용 15,000원
카드혜택: 월납요금(공과금) 10% 할인, 편의점 10% 할인
상세페이지: https://www.card-gorilla.com/card/detail/13
추천이유: 공과금과 편의점 할인율이 10%로 매우 높아 1인 가구의 고정 지출을 줄이는 데 효과적입니다. 연회비 또한 15,000원으로 원하시는 조건에 부합합니다.
====================
[카드 2]
카드사: 삼성카드
카드이름: 삼성 iD SELECT ALL 카드
연회비: 국내 20,000원 / 해외 20,000원
카드혜택: 아파트 관리비/통신 10% 할인, 편의점 7% 할인
상세페이지: 확인 필요
추천이유: 통신비와 관리비 등 고정 지출에서 10% 할인을 제공하여 자취생에게 유리합니다. 편의점 등 일상 영역에서도 7% 할인을 받을 수 있습니다.
====================
[카드 3]
카드사: 우리카드
카드이름: 카드의정석 SHOPPING+
연회비: 국내 10,000원 / 해외 12,000원
카드혜택: 온라인/오프라인 쇼핑 10% 청구할인
상세페이지: 확인 필요
추천이유: 자취에 필요한 물품을 온/오프라인 쇼핑으로 구매하실 때 범용적인 10% 할인을 받을 수 있는 카드입니다. 연회비 부담이 적습니다.
====================
[카드 4]
카드사: 현대카드
카드이름: 현대카드ZERO MOBILE Edition2(할인형)
연회비: 확인 필요
카드혜택: 전 가맹점 할인 (상세 확인 필요)
상세페이지: 확인 필요
추천이유: 특정 가맹점이나 실적 조건에 신경 쓰기 번거로우실 때, 전 가맹점 기본 할인을 통해 편하게 사용하실 수 있는 범용 카드입니다.
        """
    },
    {
        "User_Input": "요즘 혜택 좋은 신용카드 아무거나 4개만 추천해주세요.",
        "Output":"""
[카드 1]
카드사: 현대카드
카드이름: 현대카드ZERO Edition3(할인형)
연회비: 국내전용 15,000 원, 해외겸용 15,000 원
카드혜택: 국내외 가맹점 이용 금액 0.8% 청구 할인 (전월실적 조건 및 한도 제한 없음)
상세페이지: https://www.card-gorilla.com/card/detail/2646
추천이유: 복잡한 전월 실적 조건 없이 모든 가맹점에서 기본 할인을 받을 수 있는 범용성 높은 카드라 추천했습니다. 카드 관리가 번거로운 분들에게 최적화된 무조건 할인 유형입니다.
====================
[카드 2]
카드사: 롯데카드
카드이름: LOCA 365 카드
연회비: 국내전용 20,000 원, 해외겸용 20,000 원
카드혜택: 아파트관리비, 공과금, 이동통신, 대중교통 10% 결제일 할인
상세페이지: https://www.card-gorilla.com/card/detail/2330
추천이유: 매월 발생하는 공과금, 아파트관리비, 통신비 등 고정지출 영역에서 강력한 10% 할인을 제공하여 추천했습니다. 고정비 절감 효과가 커 직장인과 1인 가구 모두에게 인기가 많습니다.
====================
[카드 3]
카드사: 삼성카드
카드이름: 삼성카드 taptap O
연회비: 국내전용 10,000 원, 해외겸용 10,000 원
카드혜택: 스타벅스 50% 할인 또는 커피전문점 30% 할인, 쇼핑 7% 할인
상세페이지: https://www.card-gorilla.com/card/detail/51
추천이유: 커피전문점 최대 50% 할인 및 온라인 쇼핑 할인을 제공하여 2030 트렌드 소비에 맞춰 추천했습니다. 본인의 소비 패턴에 맞게 혜택 패키지를 매월 변경할 수 있는 것이 장점입니다.
====================
[카드 4]
카드사: 삼성카드
카드이름: 삼성카드 & MILEAGE PLATINUM (스카이패스)
연회비: 국내전용 47,000 원, 해외겸용 49,000 원
카드혜택: 모든 가맹점 1천원당 1 마일리지 기본 적립, 백화점/주유 등 2 마일리지 특별 적립
상세페이지: https://www.card-gorilla.com/card/detail/49
추천이유: 여행을 즐기시는 분들을 위한 대한항공 마일리지 적립 특화 카드로 추천했습니다. 전월 실적 조건 없이 결제 금액 1천 원당 마일리지가 지속적으로 적립되어 마일리지 모으기에 효율적입니다.
        """
    },
    {
        "User_Input": "쓸만한 신용카드 하나 추천해주세요.",
        "Output":"""
[카드 1]
카드사: 현대카드
카드이름: 현대카드ZERO(할인형)
연회비: 확인 필요
카드혜택: 조건 없는 가맹점 기본 할인
상세페이지: 확인 필요
추천이유: 전월 실적 조건 없이 혜택을 받을 수 있어 관리가 편한 범용 카드입니다.
====================
[카드 2]
카드사: 신한카드
카드이름: 신한카드 Mr.Life
연회비: 해외겸용 15,000원
카드혜택: 월납요금(공과금) 10% 할인, 병원/약국 10% 할인
상세페이지: https://www.card-gorilla.com/card/detail/13
추천이유: 매월 발생하는 공과금, 병원비 등 일상적인 고정 지출을 방어하는 데 특화된 카드입니다.
====================
[카드 3]
카드사: 삼성카드
카드이름: 삼성 iD SELECT ALL 카드
연회비: 국내 20,000원 / 해외 20,000원
카드혜택: 디지털콘텐츠 멤버십 50% 할인, 배달앱 7% 할인
상세페이지: 확인 필요
추천이유: OTT 등의 디지털 구독과 배달앱 혜택이 강력하여 트렌디한 소비 생활에 적합합니다.
====================
[카드 4]
카드사: 삼성카드
카드이름: 삼성 iD SELECT ALL 카드
연회비: 국내 20,000원 / 해외 20,000원
카드혜택: 해외 2% 할인
상세페이지: 확인 필요
추천이유: 해외 결제 시 2% 할인을 제공하여 여행이나 해외 직구를 즐기시는 분들께 유리합니다.
        """
    }
]


# FewShot 템플릿 생성
example_template = ChatPromptTemplate([
    ("user", "{User_Input}"),
    ("ai", "{Output}")
])

# 모델 전달 템플릿 생성
fewshot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_template
)

final_template = ChatPromptTemplate([
    ("system", system_prompt),
    fewshot_prompt,
    ("user", "{input_data}")
])


# FewShot 체인 생성
fewshot_chain = final_template | model | StrOutputParser()

print("첫번째 고객\n", input_data1)
print()
print("답변:\n", fewshot_chain.invoke(input_data1))
print()
print("두번째 고객\n", input_data2)
print()
print("답변:\n", fewshot_chain.invoke(input_data2))
print()
print("세번째 고객\n", input_data3)
print()
print("답변:\n", fewshot_chain.invoke(input_data3))
print()
print("네번째 고객\n", input_data4)
print()
print("답변:\n", fewshot_chain.invoke(input_data4))
print()
print("다섯번째 고객\n", input_data5)
print()
print("답변:\n", fewshot_chain.invoke(input_data5))

# 5. RAG 모델 구현

- AI의 프롬프트 인식 및 수행 능력을 향상시키기 위해 HTML 트리 구조로 프롬프트 재구성

- 정보 부족 모드(;Fallback Logic) 임시 제거: 주요 지시 사항과 일부 충돌 + 복잡한 프롬프트에 대한 AI의 수행 능력 및 답변 품질 저하 + 토큰 최적화
    - 정보가 불충분한 상황에서는 "확인 필요"를 출력하도록 지시해 할루시네이션 방지

In [ ]:
path1 = "./data/rag_chunks.jsonl"
path2 = "./data/card_ranking_for_rag.jsonl"

docs = []
with open(path1, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        docs.append(
            Document(
                page_content=str(line)
                )
            )

with open(path2, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        docs.append(
            Document(
                page_content=str(line)
                )
            )

splitter = CharacterTextSplitter(
    separator="", # 분할 기준
    chunk_size=600,
    chunk_overlap=100
)

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# vectorstore = Chroma.from_documents(
#     documents=docs,
#     embedding=embedding,
#     persist_directory="./Chroma",
#     collection_name="cardGorilla"
# )

vectorstore = Chroma(
    embedding_function=embedding,
    persist_directory="./Chroma",
    collection_name="cardGorilla"
)

user_prompt = """\
아래 사용자의 정보를 보고 그 상황에 맞는 알맞은 카드를 추천해주세요.

[사용자 정보]
{input_data}

[카드 정보]
{context}\
"""



retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 50,
        "fetch_k": 1000,
        "lambda_mult": 0.8
    }
)
result1 = retriever.invoke(input_data1)
result2 = retriever.invoke(input_data2)
result3 = retriever.invoke(input_data3)
result4 = retriever.invoke(input_data4)
result5 = retriever.invoke(input_data5)



rag_system_prompt = """\
<System_Prompt>
    <Role_Assignment>
        당신은 "한국 신용카드 추천 챗봇"이다. 반드시 제공된 <context> 데이터와 <user_prompt>에 포함된 정보에 기반하여, 유저에게 가장 적합한 카드 4종의 추천을 단 한 번의 응답으로 제공해야 한다.
    </Role_Assignment>

    <Absolute_Rules>
    <!-- AI instruction: Follow these internal rules STRICTLY for generating outputs. -->
    1) 질문 금지: 어떤 경우에도 사용자에게 질문/역질문/확인 요청을 하지 않는다.
    2) RAG 기반 추천: 추천 카드는 반드시 <context> 내에 존재하는 카드 중에서만 선택한다. 절대 존재하지 않는 카드를 창작해 답변하지 않는다. 카드 정보 또한 <context> 내에 존재하는 정보만을 사용한다.
        - 유저가 명시한 선호/제약(연회비 상한, 혜택 카테고리, 카드사 선호/비선호, 해외/국내, 할인/적립/마일리지, 전월실적 부담 등)이 있으면 최우선 반영한다.
    3) 출력 포맷 강제: 출력은 <Output_Format>에서 지정된 카드 블록과 구분선만 허용한다. 인사말, 서론, 요약, 면책 조항, 추가 안내 등의 출력을 금지한다.
    4) 카드 수: 반드시 4개만 출력한다(초과/미만 금지).
    5) 사실성 유지: <context> 내에 한 카드의 이름을 찾았다면 해당 카드의 모든 혜택을 끝까지 검색하여 통합한다. 한 카드의 모든 카드 혜택을 가져와 출력한다.
    6) 추천이유 규칙: 추천 이유는 5줄 이내(줄바꿈 기준)로 작성한다. 질문 및 과장된 표현(무조건, 최고, 절대 등) 사용을 금지한다.
    7) 문체 지정: 한국어 존댓말을 사용하며, 문장 종결은 “~니다.” 체를 사용한다.
    </Absolute_Rules>

    <Output_Format>
    출력 형식(정확히 이 구조 + 카드 사이 구분선 필수)
    [카드 1]
    카드사: ...
    카드이름: ...
    연회비: ...
    카드혜택: ...
    상세페이지: ...
    추천이유: ...
    (추천이유는 최대 5가지)
    ====================
    [카드 2]
    카드사: ...
    카드이름: ...
    연회비: ...
    카드혜택: ...
    상세페이지: ...
    추천이유: ...
    (추천이유는 최대 5줄)
    ====================
    [카드 3]
    카드사: ...
    카드이름: ...
    연회비: ...
    카드혜택: ...
    상세페이지: ...
    추천이유: ...
    (추천이유는 최대 5줄)
    ====================
    [카드 4]
    카드사: ...
    카드이름: ...
    연회비: ...
    카드혜택: ...
    상세페이지: ...
    추천이유: ...
    (추천이유는 최대 5줄)
    </Output_Format>

    <Few_Shot_Examples>
    <Example_A type="구체적인 조건이 있는 경우">
        <User_Query_1> "저는 20대 취준생입니다. 매일 대중교통으로 학원에 가고, 식비는 주로 편의점에서 해결해요. 전월실적 부담이 적고 대중교통, 편의점 할인이 되는 카드를 추천해주세요."</User_Query>
        <AI_Response_1>
            [카드 1]
            카드사: 신한카드
            카드이름: 신한카드 Mr.Life
            연회비: 해외겸용 15,000 원
            카드혜택: 편의점 10% 할인, 병원/약국 10% 할인, 공과금 10% 할인
            상세페이지: https://www.card-gorilla.com/card/detail/13
            추천이유: 편의점 결제 시 10% 할인을 제공하여 식비 절감에 유리해 추천했습니다. 공과금과 통신비 할인도 포함되어 자취하는 취업준비생의 고정지출 방어에 적합합니다. 전월실적 30만 원으로 비교적 유지 부담이 적은 편입니다.
            ====================
            [카드 2]
            카드사: 삼성카드
            카드이름: 삼성카드 taptap O
            연회비: 국내전용 10,000 원, 해외겸용 10,000 원
            카드혜택: 대중교통/택시 10% 결제일할인, 편의점 7% 결제일할인(옵션), 이동통신 10% 할인
            상세페이지: https://www.card-gorilla.com/card/detail/51
            추천이유: 매일 이용하는 대중교통 요금을 10% 할인받을 수 있어 통학 부담을 줄여주어 추천했습니다. 라이프스타일 패키지 선택을 통해 편의점 7% 할인 혜택을 챙길 수 있습니다. 연회비가 1만 원으로 저렴하여 20대가 발급받기 좋습니다.
            ====================
            [카드 3]
            카드사: KB국민카드
            카드이름: KB국민 My WE:SH 카드
            연회비: 국내전용 15,000 원, 해외겸용 15,000 원
            카드혜택: KB Pay 결제 시 음식점/편의점 10% 할인, 이동통신 10% 할인
            상세페이지: https://www.card-gorilla.com/card/detail/2441
            추천이유: 식비를 아끼기 위해 편의점을 자주 이용하는 패턴에 맞춰 편의점 10% 할인이 제공되어 추천했습니다. 앱 결제(KB Pay)를 활용하면 생활 전반에서 할인 혜택을 누리기 좋습니다.
            ====================
            [카드 4]
            카드사: 현대카드
            카드이름: 현대카드ZERO Edition3(할인형)
            연회비: 국내전용 15,000 원, 해외겸용 15,000 원
            카드혜택: 국내외 가맹점 이용 금액 0.8% 청구 할인 (전월실적 조건 및 한도 제한 없음)
            상세페이지: https://www.card-gorilla.com/card/detail/2646
            추천이유:
            전월실적 조건이나 한도 제한이 전혀 없어 소득이 일정하지 않은 취준생도 부담 없이 사용할 수 있어 추천했습니다. 대중교통, 편의점을 포함해 어디서든 기본 할인이 적용되어 관리가 편리합니다.
        </AI_Response_1>

        <User_Query_2> "자취하는 대학생인데 공과금이랑 편의점 할인이 많이 되는 카드를 원해요. 연회비는 2만원 이하였으면 좋겠어요."</User_Query_2> 
        <AI_Response_2>
            [카드 1]
            카드사: 신한카드
            카드이름: 신한카드 Mr.Life
            연회비: 해외겸용 15,000원
            카드혜택: 월납요금(공과금) 10% 할인, 편의점 10% 할인
            상세페이지: https://www.card-gorilla.com/card/detail/13
            추천이유: 공과금과 편의점 할인율이 10%로 매우 높아 1인 가구의 고정 지출을 줄이는 데 효과적입니다. 연회비 또한 15,000원으로 원하시는 조건에 부합합니다.
            ====================
            [카드 2]
            카드사: 삼성카드
            카드이름: 삼성 iD SELECT ALL 카드
            연회비: 국내 20,000원 / 해외 20,000원
            카드혜택: 아파트 관리비/통신 10% 할인, 편의점 7% 할인
            상세페이지: 실제 데이터가 들어간 완벽한 답변
            추천이유: 통신비와 관리비 등 고정 지출에서 10% 할인을 제공하여 자취생에게 유리합니다. 편의점 등 일상 영역에서도 7% 할인을 받을 수 있습니다.
            ====================
            [카드 3]
            카드사: 우리카드
            카드이름: 카드의정석 SHOPPING+
            연회비: 국내 10,000원 / 해외 12,000원
            카드혜택: 온라인/오프라인 쇼핑 10% 청구할인
            상세페이지: 실제 데이터가 들어간 완벽한 답변
            추천이유: 자취에 필요한 물품을 온/오프라인 쇼핑으로 구매하실 때 범용적인 10% 할인을 받을 수 있는 카드입니다. 연회비 부담이 적습니다.
            ====================
            [카드 4]
            카드사: 현대카드
            카드이름: 현대카드ZERO MOBILE Edition2(할인형)
            연회비: 실제 데이터가 들어간 완벽한 답변
            카드혜택: 전 가맹점 할인 (실제 데이터가 들어간 완벽한 답변)
            상세페이지: 실제 데이터가 들어간 완벽한 답변
            추천이유: 특정 가맹점이나 실적 조건에 신경 쓰기 번거로우실 때, 전 가맹점 기본 할인을 통해 편하게 사용하실 수 있는 범용 카드입니다.
        </AI_Response_2>
    </Example_A>
    </Few_Shot_Examples>
</System_Prompt>\
"""

rag_template = ChatPromptTemplate([
    ("system", rag_system_prompt),
    ("user", user_prompt)
])

rag_chain = rag_template | model | StrOutputParser()

print("첫번째 고객\n", input_data4)
print()
print("답변:\n", rag_chain.invoke({"input_data": input_data4, "context": result4}))

## 5.1 모델 테스트 결과
 -
 -
 -
 

# 6. 프롬프트 튜닝

- AI의 프롬프트 인식 능력 + 시스템 프롬프트에 소모되는 token 절감을 위해 영문 프롬프트로 번역

- 내부 사고 프로세스 (Internal CoT) 추가: 
    1. 유저의 입력에서 특정 키워드 (선호/제외, 조건/제약, 원하는 혜택 등) 추출
    2. 키워드를 바탕으로 context 내에서 카드 필터링
    3. 조건에 가장 부합한 상위 4카드 추천
- 분리되어 있던 카드 기본 정보 데이터와 랭킹 정보를 단일 청크로 제공
    - -> context로 제공한 시계열 랭킹 데이터를 추천 이유 제시에 참고하도록 지시
    
- 추천 이유를 '최대 5줄'에서 '최대 5가지'로 제시하도록 지시 변경

In [ ]:
import re

path = "./data/rag_chunks_with_rankings.jsonl"

docs = []
with open(path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        docs.append(
            Document(
                page_content=str(line)
                )
            )

splitter = CharacterTextSplitter(
    separator="", # 분할 기준
    chunk_size=600,
    chunk_overlap=100
)

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# vectorstore = Chroma.from_documents(
#     documents=docs,
#     embedding=embedding,
#     persist_directory="./Chroma",
#     collection_name="cardGorilla2"
# )

vectorstore = Chroma(
    embedding_function=embedding,
    persist_directory="./Chroma",
    collection_name="cardGorilla2"
)

user_prompt = """\
아래 사용자의 정보를 보고 그 상황에 맞는 알맞은 카드를 추천해주세요.

[사용자 정보]
{input_data}

[카드 정보]
{context}\
"""



retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 30,
        "fetch_k": 500,
        "lambda_mult": 0.8
    }
)
result1 = retriever.invoke(input_data1)
result2 = retriever.invoke(input_data2)
result3 = retriever.invoke(input_data3)
result4 = retriever.invoke(input_data4)
result5 = retriever.invoke(input_data5)



rag_system_prompt = """\
<System_Prompt>
    <Role_Assignment>
        You are a "Korean Credit Card Recommendation Chatbot". You MUST provide exactly 4 suitable card recommendations to the user in a single response, strictly based on the provided <context> data and the information in the <user_prompt>.
    </Role_Assignment>

    <Core_Directives>
    1. No Questions: DO NOT ask questions, counter-questions, or request clarification from the user under any circumstances.
    2. RAG-Based Recommendation: ONLY select cards that exist within the <context>. NEVER hallucinate or invent non-existent cards. Use ONLY the information provided in the <context>.
    3. Output ALL Benefits: You MUST extract and list ALL the card benefits mentioned in the <context>. Do not summarize them into just one benefit.
    4. Internal CoT (Chain of Thought): Before generating the final card recommendations, you MUST process the user request internally using the <Thinking_Process> block.
        - Step 1 (Keyword Extraction): Extract keywords from the user's input (e.g., Preferred/Excluded card companies, Annual fee limits, Previous month performance constraints, Target benefit categories).
        - Step 2 (Filtering): Filter the cards in the <context> based on the extracted keywords.
        - Step 3 (Selection): Select the top 4 most appropriate cards that match the criteria.
    5. Strict Output Format: You MUST exactly follow the <Output_Format> structure. Output the <Thinking_Process> block first, followed by the card blocks and separators. DO NOT output greetings, introductions, summaries, or disclaimers.
    6. Exact Card Count: You MUST output exactly 4 cards. Neither more nor less.
    7. Recommendation Reason Rules:
        - Provide the recommendation reasons as a numbered list (1, 2, 3...) with a MAXIMUM of 5 items.
        - You MUST utilize the card's ranking data from the <context> and include it as one of the reasons (e.g., "이 카드는 카드고릴라 랭킹 X위로 검증된 인기 카드입니다.").
        - DO NOT use exaggerated expressions (e.g., "무조건", "최고", "절대").
    8. Factual Accuracy: If the annual fee, previous month performance (전월실적), benefits, or URL in the <context> is unclear or missing, strictly write "확인 필요" for that specific field.
    9. Tone and Language: Strictly use polite Korean, ending sentences with the "~니다" form. DO NOT contain English in responses.
    </Core_Directives>

    <Output_Format>
    <Thinking_Process>
    - 사용자 키워드 추출: [선호 카드사: ...], [전월실적 조건: ...], [연회비 조건: ...], [주요 혜택: ...] 등
    - <context> 필터링 및 추천: [선정된 4가지 카드 이름과 매칭 이유 간략히]
    </Thinking_Process>
    [카드 1]
    카드사: ...
    카드이름: ...
    연회비: ...
    전월실적: ...
    카드혜택: ... (컨텍스트 내 모든 혜택 나열)
    상세페이지: ...
    추천이유:
    1. ... (반드시 랭킹 정보 포함)
    2. ...
    3. ...
    4. ...
    5. ...
    (최대 5가지 항목으로 작성)
    ====================
    [카드 2]
    ... (위와 동일 구조) ...
    ====================
    [카드 3]
    ... (위와 동일 구조) ...
    ====================
    [카드 4]
    ... (위와 동일 구조) ...
    </Output_Format>

    <Few_Shot_Examples>
    <Example>
        <User_Query> "20대 취준생입니다. 매일 대중교통으로 학원에 가고 편의점을 자주 가요. 전월실적이 30만원 이하로 낮았으면 좋겠고, 신한카드나 삼성카드를 선호합니다."</User_Query>
        <AI_Response>
            <Thinking_Process>
            - 사용자 키워드 추출: [주요 혜택: 대중교통, 편의점], [전월실적 조건: 30만원 이하], [선호 카드사: 신한카드, 삼성카드]
            - 컨텍스트 필터링 및 카드 선정:
              1. 신한카드 Mr.Life: 편의점 혜택 존재, 실적 30만원, 신한카드 충족 (랭킹 1위)
              2. 삼성카드 taptap O: 대중교통/편의점 혜택 존재, 실적 30만원, 삼성카드 충족 (랭킹 5위)
              3. 신한카드 Deep Dream: 전월실적 조건 없음, 편의점 적립, 신한카드 충족 (랭킹 12위)
              4. 삼성 iD SELECT ALL 카드: 편의점 혜택, 삼성카드 충족 (랭킹 2위)
            </Thinking_Process>

            [카드 1]
            카드사: 신한카드
            카드이름: 신한카드 Mr.Life
            연회비: 해외겸용 15,000 원
            전월실적: 30만 원 이상
            카드혜택: 편의점 10% 할인, 병원/약국 10% 할인, 세탁소 10% 할인, 오후 9시~오전 9시 온라인쇼핑 10% 할인, 전기/도시가스/통신요금 10% 할인
            상세페이지: https://www.card-gorilla.com/card/detail/13
            추천이유:
            1. 2026년 2월 기준 카드고릴라 랭킹 1위를 차지할 만큼 많은 사람에게 검증된 인기 카드입니다.
            2. 편의점 결제 시 10% 할인을 제공하여 취업준비생의 식비 절감에 매우 유리합니다.
            3. 전월실적 조건이 30만 원으로 설정되어 있어 유지 부담이 적습니다.
            4. 통신요금 및 공과금 10% 할인이 포함되어 자취 고정비 방어에 탁월합니다.
            5. 사용자가 선호하는 신한카드사의 상품입니다.
            ====================
            [카드 2]
            카드사: 삼성카드
            카드이름: 삼성카드 taptap O
            연회비: 국내전용 10,000 원, 해외겸용 10,000 원
            전월실적: 30만 원 이상
            카드혜택: 대중교통/택시 10% 결제일할인, 스타벅스 50% 할인, 편의점 7% 결제일할인(옵션), 이동통신 10% 할인, CGV/롯데시네마 5,000원 할인
            상세페이지: https://www.card-gorilla.com/card/detail/51
            추천이유:
            1. 2026년 2월 기준 카드고릴라 랭킹 6위에 올라 있는 선호도 최상위 카드입니다.
            2. 매일 이용하시는 대중교통 요금을 10% 할인받을 수 있어 통학 부담을 크게 덜어줍니다.
            3. 라이프스타일 패키지 선택을 통해 편의점 7% 할인 혜택을 챙기실 수 있습니다.
            4. 연회비가 1만 원으로 저렴하고 전월실적이 30만 원이라 관리가 수월합니다.
            5. 사용자가 선호하는 삼성카드사의 상품입니다.
            ====================
            [카드 3]
            카드사: 신한카드
            카드이름: 신한카드 처음(ANNIVERSE)
            연회비: 국내전용 15,000 원, 해외겸용 18,000 원
            전월실적: 전월실적 30만 원 이상
            카드혜택: 음식점·카페·편의점·온라인쇼핑 5% 마이신한포인트 적립, 생활·여행·패션 5% 마이신한포인트적립, 통신 10% , OTT 15%, 멤버십 20% 마이신한포인트 적립, 소비관리 보너스(계획소비 적립, 즉시결제 적립)
            상세페이지: 확인 필요
            추천이유:
            1. 2026년 2월 기준 카드고릴라 랭킹 12위에 위치한 대표적인 적립형 카드입니다.
            2. 편의점, 온라인쇼핑 등 20대 대학생이 자주 소비하는 분야에서 5% 포인트 적립을 제공합니다.
            3. 소비관리 보너스(계획소비 적립, 즉시결제 적립) 혜택이 있어 범용성이 뛰어납니다.
            4. 사용자가 명시한 신한카드사의 상품입니다.
            ====================
            [카드 4]
            카드사: 삼성카드
            카드이름: 삼성 iD SELECT ALL 카드
            연회비: 국내 20,000 원, 해외 20,000 원
            전월실적: 확인 필요
            카드혜택: 아파트 관리비/통신 10% 할인, 편의점 7% 할인, 배달앱 7% 할인, 디지털콘텐츠 멤버십 50% 할인
            상세페이지: 확인 필요
            추천이유:
            1. 2026년 2월 기준 카드고릴라 랭킹 2위를 차지한 대세 카드입니다.
            2. 편의점 7% 할인을 비롯해 배달앱 할인까지 제공되어 식비 관리에 유리합니다.
            3. OTT 등 디지털 구독 50% 할인이 포함되어 있어 20대 라이프스타일에 부합합니다.
            4. 통신비 10% 할인을 통해 추가적인 고정지출 방어가 가능합니다.
            5. 선호하시는 삼성카드 라인업 중 최근 가장 트렌디한 혜택을 담고 있습니다.
        </AI_Response>
    </Example>
    </Few_Shot_Examples>
</System_Prompt>
"""

rag_template = ChatPromptTemplate([
    ("system", rag_system_prompt),
    ("user", user_prompt)
])

model = "gpt-5-nano"
model = ChatOpenAI(model=model, temperature=0.8)

rag_chain = rag_template | model | StrOutputParser()

def clean_output(text):
    return re.sub(r'<Thinking_Process>.*?</Thinking_Process>', '', text, flags=re.DOTALL).strip()

# n명의 데이터를 하나의 리스트로 묶기 (현재 5명)
test_customers = [
    (input_data1, result1),
    (input_data2, result2),
    (input_data3, result3),
    (input_data4, result4),
    (input_data5, result5)
]

# for문(반복문)을 사용해 한 번에 처리
for i, (user_input, ctx) in enumerate(test_customers, 1):
    print(f"{i}번째 고객\n{user_input}")
    
    # 모델 응답 생성
    raw_ans = rag_chain.invoke({"input_data": user_input, "context": ctx})
    
    # 정제 후 출력
    print("\n답변:\n", clean_output(raw_ans))
    print("\n" + "="*50 + "\n")

## 6.1 모델 테스트 결과

# 7. 최종 프롬프트

- context 내에 분리되어 있던 카드 기본 정보와 랭킹 데이터 통합 -> 랭킹 정보는 있지만 카드 기본 정보는 없는 카드 필터링 구현
- 카드 혜택이 전체가 아닌 일부만 출력되는 문제 해결 -> 카드 혜택 전부 출력 시 사용자 가독성을 높이기 위해 출력 포맷 일부 수정

## 7.1 코드 전처리부

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
import json
import math
import re

def is_valid(val):
    """값이 유효한지(None, NaN, 빈 문자열 아님) 엄격하게 확인"""
    if val is None: 
        return False
    if isinstance(val, float) and math.isnan(val): 
        return False
    if isinstance(val, str) and (val.strip() == "" or val.strip().lower() == "nan"): 
        return False
    return True

def to_int_safe(x):
    """랭킹을 '파손 없이' 안전하게 int 변환 시도(실패 시 None)"""
    try:
        # "12", 12, 12.0 같은 케이스 대응
        if isinstance(x, bool):
            return None
        if isinstance(x, (int,)):
            return int(x)
        if isinstance(x, float):
            if math.isnan(x):
                return None
            return int(x)
        if isinstance(x, str):
            s = x.strip()
            if s == "" or s.lower() == "nan":
                return None
            # "12위" 같은 문자열도 숫자만 뽑아 시도(원본은 따로 보관)
            m = re.search(r"\d+", s)
            return int(m.group()) if m else None
    except:
        return None
    return None

path = "./data/rag_chunks_with_rankings.jsonl"
consolidated = {}

with open(path, "r", encoding="utf-8") as f:
    for line in f:
        clean_line = line.replace(": NaN", ": null").replace(": nan", ": null")
        try:
            data = json.loads(clean_line)
        except:
            continue

        name = data.get("card_name")
        comp = data.get("company")
        if not name or not comp:
            continue

        card_id = f"{name}_{comp}"

        # ✅ [추가] 랭킹 저장 구조 포함
        if card_id not in consolidated:
            consolidated[card_id] = {
                "card_name": name,
                "company": comp,
                "annual_fee": None,
                "performance": None,
                "url": None,
                "benefits": set(),

                # ---- ranking fields (원본 보존 중심) ----
                "rank_best_num": None,              # 가장 좋은(최소) 랭킹 숫자
                "rank_best_raw": None,              # 그때의 원본(raw) 값(문자열)
                "rank_raw_set": set(),              # 관측된 모든 원본 랭킹 값(파손 방지)
                "rank_by_category_raw": {},         # 카테고리별 첫 관측 raw 랭킹(덮어쓰기 방지)
            }

        card = consolidated[card_id]

        # ✅ [추가] 랭킹 추출/통합 (원본 파손 방지 + 최소랭킹 산출)
        r = data.get("rank")
        if is_valid(r):
            raw = str(r).strip()
            if raw:
                card["rank_raw_set"].add(raw)

                # 카테고리별 raw 저장(이미 있으면 덮어쓰지 않음)
                cat = data.get("category_norm", "기타")
                if is_valid(cat):
                    cat = str(cat).strip() or "기타"
                else:
                    cat = "기타"
                if cat not in card["rank_by_category_raw"]:
                    card["rank_by_category_raw"][cat] = raw

                # best rank 계산(숫자 변환 가능할 때만)
                r_num = to_int_safe(r)
                if r_num is not None:
                    if (card["rank_best_num"] is None) or (r_num < card["rank_best_num"]):
                        card["rank_best_num"] = r_num
                        card["rank_best_raw"] = raw

        # 3. 상세페이지 URL (이미 있으면 덮어쓰지 않음)
        current_url = data.get("detail_url")
        if is_valid(current_url) and not is_valid(card["url"]):
            card["url"] = current_url.strip()

        # 4. 연회비 (metrics 우선)
        metrics = data.get("metrics", {})
        if not is_valid(card["annual_fee"]):
            o_fee = metrics.get("fee_overseas_won")
            d_fee = metrics.get("fee_domestic_won")
            if is_valid(o_fee):
                card["annual_fee"] = f"해외 {int(o_fee):,}원"
            elif is_valid(d_fee):
                card["annual_fee"] = f"국내 {int(d_fee):,}원"

        # 5. 전월실적 (conditions 우선)
        if not is_valid(card["performance"]):
            p_won = data.get("conditions", {}).get("prev_month_won")
            if is_valid(p_won):
                card["performance"] = f"{int(p_won):,}원 이상"

        # 6. 혜택 정보 통합
        text_val = data.get("text", "")
        if "혜택 상세:" in text_val:
            benefit_detail = text_val.split("혜택 상세:")[1].split("|")[0].strip()
            category = data.get("category_norm", "기타")
            card["benefits"].add(f"[{category}] {benefit_detail}")

# 7. Document 객체로 변환 (✅ 랭킹을 컨텍스트에 포함)
docs = []
for info in consolidated.values():
    # best rank 표시: 숫자(best)가 있으면 그걸 우선, 없으면 raw 중 하나라도 있으면 raw 기반으로 표기
    if info["rank_best_num"] is not None:
        rank_text = f"{info['rank_best_num']}위"
    elif len(info["rank_raw_set"]) > 0:
        # raw가 여러 개면 그대로 모두 보관되어 있으니, 컨텍스트에는 합쳐서 노출
        rank_text = " / ".join(sorted(info["rank_raw_set"])) + " (원본)"
    else:
        rank_text = "확인 필요"

    # 카테고리별 랭킹(원본 그대로)도 함께 제공(원하면 프롬프트에서 활용 가능)
    if info["rank_by_category_raw"]:
        by_cat = ", ".join([f"{k}:{v}" for k, v in sorted(info["rank_by_category_raw"].items())])
    else:
        by_cat = "확인 필요"

    content = (
        f"카드명: {info['card_name']} ({info['company']})\n"
        f"카드고릴라랭킹: {rank_text}\n"
        f"카테고리별랭킹(원본): {by_cat}\n"
        f"연회비: {info['annual_fee'] if is_valid(info['annual_fee']) else '확인 필요'}\n"
        f"전월실적: {info['performance'] if is_valid(info['performance']) else '확인 필요'}\n"
        f"전체혜택:\n- " + "\n- ".join(sorted(list(info['benefits']))) + "\n"
        f"상세페이지: {info['url'] if is_valid(info['url']) else '확인 필요'}"
    )
    docs.append(Document(page_content=content))

## 7.2 실행부

In [ ]:
# input_data(페르소나)
input_data1 = """\
전월실적 부담 적고 교통비 할인 많이 되는 신용카드 알려주세요. 연회비 2만원 이하면 좋겠어요.\
"""

input_data2 = """\
OTT 구독이랑 배달앱 할인 잘 되는 카드 알려주세요. 삼성카드나 현대카드면 좋겠어요.\
"""

input_data3 = """\
전월실적 부담 없이 모든 곳에서 기본 할인 받을 수 있는 카드 있나요?"\
"""

input_data4 = """\
주유 할인이랑 아파트 관리비 할인 되는 카드 추천해주세요. 전월실적 높아도 괜찮아요, 월 사용액이 300만원 넘습니다.\
"""

input_data5 = """\
병원, 약국, 공과금 할인 잘 되는 카드 찾고 있어요. KB국민카드나 신한카드로 추천해주세요.\
"""

In [ ]:
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# vectorstore = Chroma.from_documents(
#     documents=docs,
#     embedding=embedding,
#     persist_directory="./Chroma",
#     collection_name="cardGorilla10"
# )
# vectorstore.persist()

vectorstore = Chroma(
    embedding_function=embedding,
    persist_directory="./Chroma",
    collection_name="cardGorilla10"
)

retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 50,
        "fetch_k": 500,
        "lambda_mult": 0.6
    }
)
result1 = retriever.invoke(input_data1)
result2 = retriever.invoke(input_data2)
result3 = retriever.invoke(input_data3)
result4 = retriever.invoke(input_data4)
result5 = retriever.invoke(input_data5)


rag_system_prompt = """\
<System_Prompt>
    <Role_Assignment>
        You are "Korean Credit Card Recommendation Chatbot". You MUST provide exactly 4 suitable card recommendations to the user in a single response, strictly based on the provided <context> data and the information in the <user_prompt>.
    </Role_Assignment>

    <Core_Directives>
    1. No Questions: DO NOT ask questions, counter-questions, or request clarification from user under any circumstances.
    2. RAG-Based Recommendation: ONLY select cards that exist within <context>. NEVER hallucinate or invent non-existent cards. Use ONLY the information provided in the <context>.
    3. Output ALL Benefits: You MUST extract and list ALL card benefits mentioned in <context>. Do not summarize them into just one benefit.
    4. Exclude Invalid Cards (Strict): NEVER recommend card if its benefit details are missing, or if it ONLY contains ranking information in <context>. A valid recommendation MUST contain actual card benefits.
    5. Internal CoT (Chain of Thought): Before generating final card recommendations, you MUST process user request internally using <Thinking_Process> block.
        - Step 1 (Keyword Extraction): Extract keywords from user's input (Preferred/Excluded card companies, Annual fee limits, Previous month performance constraints, Target benefit categories, or etc.).
        - Step 2 (Filtering): Filter out cards that lack benefit information. Then match remaining cards with keywords.
        - Step 3 (Selection): Select top 4 most appropriate cards that match the criteria.
    6. Strict Output Format: You MUST exactly follow <Output_Format> structure. DO NOT output greetings, introductions, summaries, or disclaimers outside of the specified blocks.
    7. Exact Card Count: You MUST output exactly 4 cards. Neither more nor less.
    8. Recommendation Reason Rules:
        - Provide recommendation reasons as numbered list (1, 2, 3...) with MAXIMUM of 5 items.
        - You MUST utilize ranking info if it exists in <context> and include it as one of the reasons (e.g., "이 카드는 최근 YYYY년 MM월 기준 카드고릴라 랭킹 X위이며, 월간 Top100에 총 X회 진입한 검증된 인기 카드입니다.").
        - DO NOT use exaggerated expressions (e.g., "무조건", "최고", "절대").
    9. Factual Accuracy: If annual fee, previous month performance (전월실적), benefits, or URL in <context> is unclear or missing, **strictly write "확인 필요" for that specific field**.
    10. Tone and Language: Strictly use polite Korean, ending sentences with "~니다" form. DO NOT contain English in responses.
    </Core_Directives>

    <Output_Format>
    <Thinking_Process>
    - 사용자 키워드 추출: [선호 카드사: ...], [전월실적 조건: ...], [연회비 조건: ...], [주요 혜택: ...] 등
    - <context> 필터링 및 추천: [선정된 4가지 카드 이름과 매칭 이유 간략히]
    </Thinking_Process>
    [카드 1]
    카드사: ...
    카드이름: ...
    연회비: ...
    전월실적: ...
    카드혜택: 
    - [혜택 분야] 혜택 상세
    - [혜택 분야] 혜택 상세
    ... (컨텍스트 내 모든 카드 혜택 나열, 각 혜택 마다 개행 후 나열) ...
    상세페이지: ...
    추천이유:
    1. ... (반드시 랭킹 정보 포함하면서 기준 날짜도 함께 포함)
    2. ...
    3. ...
    4. ...
    5. ...
    (최대 5가지 항목으로 작성)
    ====================
    [카드 2]
    ... (위와 동일 구조) ...
    ====================
    [카드 3]
    ... (위와 동일 구조) ...
    ====================
    [카드 4]
    ... (위와 동일 구조) ...
    </Output_Format>

    <Few_Shot_Examples>
    <Example>
        <User_Query> "20대 취준생입니다. 매일 대중교통으로 학원에 가고 편의점을 자주 가요. 전월실적이 30만원 이하로 낮았으면 좋겠고, 신한카드나 삼성카드를 선호합니다."</User_Query>
        <AI_Response>
            <Thinking_Process>
            - 사용자 키워드 추출: [주요 혜택: 대중교통, 편의점], [전월실적 조건: 30만원 이하], [선호 카드사: 신한카드, 삼성카드]
            - 컨텍스트 필터링 및 카드 선정:
              1. 신한카드 Mr.Life: 편의점 혜택 존재, 실적 30만원, 신한카드 충족 (랭킹 1위)
              2. 삼성카드 taptap O: 대중교통/편의점 혜택 존재, 실적 30만원, 삼성카드 충족 (랭킹 5위)
              3. 신한카드 Deep Dream: 전월실적 조건 없음, 편의점 적립, 신한카드 충족 (랭킹 12위)
              4. 삼성 iD SELECT ALL 카드: 편의점 혜택, 삼성카드 충족 (랭킹 2위)
            </Thinking_Process>

            [카드 1]
            카드사: 신한카드
            카드이름: 신한카드 Mr.Life
            연회비: 해외겸용 15,000 원
            전월실적: 30만 원 이상
            카드혜택:
            - [마트/편의점] 편의점 10% 할인
            - [의료/교육] 병원/약국 10% 할인
            - [공과금/보험 (생활)] 세탁소 10% 할인
            - [쇼핑 (온라인쇼핑)] 온라인쇼핑 10% 할인
            - [공과금/보험 (공과금)] 월납요금(공과금) 10% 할인서비스
            상세페이지: https://www.card-gorilla.com/card/detail/13
            추천이유:
            1. 최근 2026년 2월 기준 카드고릴라 랭킹 1위, 월간 탑100 차트에 총 18회 진입한 많은 사람에게 검증된 인기 카드입니다.
            2. 편의점 결제 시 10% 할인을 제공하여 취업준비생의 식비 절감에 매우 유리합니다.
            3. 전월실적 조건이 30만 원으로 설정되어 있어 유지 부담이 적습니다.
            4. 통신요금 및 공과금 10% 할인이 포함되어 자취 고정비 방어에 탁월합니다.
            5. 사용자가 선호하는 신한카드사의 상품입니다.
            ====================
            [카드 2]
            카드사: 삼성카드
            카드이름: 삼성카드 taptap O
            연회비: 국내전용 10,000 원, 해외겸용 10,000 원
            전월실적: 30만 원 이상
            카드혜택:
            - [교통 (대중교통)] 대중교통/택시 10% 결제일할인
            - [문화/여가 (영화)] CGV 및 롯데시네마 5,000원 결제일할인
            - [통신 (통신)] 이동통신요금 10% 할인
            - [여행/항공/숙박 (해외)] 해외 1.3% 적립
            상세페이지: https://www.card-gorilla.com/card/detail/51
            추천이유:
            1. 월간 탑100 차트에 총 56회 진입했으며, 최근 2026년 2월 기준 카드고릴라 랭킹 6위에 올라 있는 선호도 최상위 카드입니다.
            2. 매일 이용하시는 대중교통 요금을 10% 할인받을 수 있어 통학 부담을 크게 덜어줍니다.
            3. 연회비가 1만 원으로 저렴하고 전월실적이 30만 원이라 관리가 수월합니다.
            4. 사용자가 선호하는 삼성카드사의 상품입니다.
            ====================
            [카드 3]
            카드사: 신한카드
            카드이름: 신한카드 처음(ANNIVERSE)
            연회비: 국내전용 15,000 원, 해외겸용 18,000 원
            전월실적: 30만 원 이상
            카드혜택:
            - [적립/캐시백 (적립)] 음식점·카페·편의점·온라인쇼핑 5% 마이신한포인트 적립
            - [공과금/보험 (생활)] 생활·여행·패션 5% 마이신한포인트 적립
            - [구독/디지털 (디지털구독)] 통신 10% , OTT 15%, 멤버십 20% 마이신한포인트 적립
            - [포인트/마일리지 적립] 소비관리 보너스(계획소비 & 즉시결제 적립)
            상세페이지: https://www.card-gorilla.com/card/detail/2759
            추천이유:
            1. 2026년 2월 기준 카드고릴라 랭킹 12위에 위치한 대표적인 적립형 카드입니다. 월간 탑100 차트에 총 16회 진입했습니다.
            2. 편의점, 온라인쇼핑 등 20대 대학생이 자주 소비하는 분야에서 5% 포인트 적립을 제공합니다.
            3. 소비관리 보너스(계획소비 적립, 즉시결제 적립) 혜택이 있어 범용성이 뛰어납니다.
            4. 사용자가 명시한 신한카드사의 상품입니다.
            ====================
            [카드 4]
            카드사: 삼성카드
            카드이름: 삼성 iD SELECT ALL 카드
            연회비: 국내 20,000 원, 해외 20,000 원
            전월실적: 40만원 이상
            카드혜택:
            - [적립/캐시백 (할인)] 아파트 관리비/통신 10% 할인
            - [적립/캐시백 (할인)] 편의점 7% 할인
            - [적립/캐시백 (할인)] 온라인쇼핑몰/의료/배달앱 7% 할인
            - [구독/디지털 (디지털구독)] 디지털콘텐츠 멤버십 50% 할인
            상세페이지: https://www.card-gorilla.com/card/detail/2885
            추천이유:
            1. 월간 탑100 차트에 총 6회 진입, 2026년 2월 기준 카드고릴라 랭킹 2위를 차지한 대세 카드입니다.
            2. 편의점 7% 할인을 비롯해 배달앱 할인까지 제공되어 식비 관리에 유리합니다.
            3. OTT 등 디지털 구독 50% 할인이 포함되어 있어 20대 라이프스타일에 부합합니다.
            4. 통신비 10% 할인을 통해 추가적인 고정지출 방어가 가능합니다.
            5. 선호하시는 삼성카드 라인업 중 최근 가장 트렌디한 혜택을 담고 있습니다.
        </AI_Response>
    </Example>
    </Few_Shot_Examples>
</System_Prompt>
"""

user_prompt = """\
아래 사용자의 정보를 보고 그 상황에 맞는 알맞은 카드를 추천해주세요.

[사용자 정보]
{input_data}

[카드 정보]
{context}\
"""

rag_template = ChatPromptTemplate([
    ("system", rag_system_prompt),
    ("user", user_prompt)
])

model = "gpt-5-nano"
model = ChatOpenAI(model=model, temperature=0.8)

rag_chain = rag_template | model | StrOutputParser()

def clean_output(text):
    return re.sub(r'<Thinking_Process>.*?</Thinking_Process>', '', text, flags=re.DOTALL).strip()

# n명의 데이터를 하나의 리스트로 묶기 (현재 5명)
test_customers = [
    (input_data1, result1),
    (input_data2, result2),
    (input_data3, result3),
    (input_data4, result4),
    (input_data5, result5)
]

# for문(반복문)을 사용해 한 번에 처리
for i, (user_input, ctx) in enumerate(test_customers, 1):
    print(f"{i}번째 고객\n{user_input}")
    
    # 모델 응답 생성
    raw_ans = rag_chain.invoke({"input_data": user_input, "context": ctx})
    
    # 정제 후 출력
    print("\n답변:\n", clean_output(raw_ans))
    print("\n" + "="*50 + "\n")

In [ ]:
print(raw_ans)

## 7.3 모델 테스트 결과